# Simulated spectral libraries

Spectral libraries are traditionally created from previously confidently identified experimental MS/MS spectra. \cite{Griss2016, Shao2016} By making use of the additional information present in these spectra as opposed to the simple theoretical spectra that are used by sequence database search engines, such as ion intensities and the presence of non-canonical fragments, spectral library search engines are often able to achieve higher identification rates. \cite{Zhang2011}

A disadvantage of spectral library searching however is that only peptides that are present in the spectral library can be identified, effectively limiting this approach to just the rediscovery of previously identified peptides. For example, the frequently used and high-quality [NIST spectral libraries](http://chemdata.nist.gov/dokuwiki/doku.php?id=peptidew:cdownload) typically cover only about 5 to 25% of the target organism's proteome. 

Similar to previous research by Yen et al. \cite{Yen2009, Yen2011} we will use a simulated spectral library to provide a full coverage of the proteome under consideration. As shown by Yen et al. \cite{Yen2009, Yen2011}, such proteome-wide simulated spectral libraries can yield equal or superior results compared to sequence database search engines.

We have used [MS<sup>2</sup>PIP](http://iomics.ugent.be/ms2pip) to generate simulated spectral libraries. \cite{Degroeve2013, Degroeve2015} MS<sup>2</sup>PIP consists of a highly accurate machine learning model that is able to predict the intensities of the most common fragment ions under CID and HCD fragmentation.

This notebook details how we have generated the simulated spectral libraries by interfacing with the MS<sup>2</sup>PIP prediction server. To view the original version of this notebook please visit the project's [code repository](https://bitbucket.org/proteinspector/ann-solo/).
All code in this notebook is written in Python 2.7. Some processing of protein and peptide sequences was done using [Pyteomics](https://pythonhosted.org/pyteomics/). \cite{Goloborodko2013}

As detailed next, we have generated simulated spectral libraries for two studies: the iPRG 2012 study \cite{Chalkley2014} and the study by Chick et al. \cite{Chick2015}

In [ ]:
# add the 'src' directory to the path so we can import modules
%load_ext autoreload
%autoreload 1

import os, sys

src_dir = os.path.join(os.getcwd(), os.pardir, 'src')
sys.path.append(src_dir)

In [ ]:
# package imports
from __future__ import print_function
import argparse
import json
import math
import operator
import sys

import requests
import tqdm
from pyteomics import fasta, mass, parser

%aimport convert_spectrast_to_spql
import convert_spectrast_to_spql

In [ ]:
# cleave proteins into peptides and simulate spectra using MS2PIP
def fasta_to_peptides(fasta_filename, min_length, max_length, protease='trypsin', missed_cleavages=0):
    """
    Read all proteins from a fasta file and generate in silico peptides.
    
    Peptides containing non-standard characters (B, J, O, U, X, Z, *) will be ignored.
    
    Args:
        fasta_filename: The fasta file containing the protein sequences.
        min_length: The minimum length of the generated peptides (inclusive).
        max_length: The maximum length of the generated peptides (inclusive).
        protease: The protease used to cleave the proteins into peptides (default: trypsin).
        missed_cleavages: The number of missed cleavages allowed (default: 0).
    
    Returns:
        A list of all (unique) in silico generated peptides.
    """
    peptides = set()
    with fasta.read(fasta_filename) as fasta_in:
        for protein in tqdm.tqdm(fasta_in, desc='Proteins processed', unit='proteins'):
            for peptide in parser.cleave(protein.sequence, parser.expasy_rules[protease], missed_cleavages, min_length):
                # ignore non-standard characters
                is_valid = True
                for disallowed_char in 'BJOUXZ*':
                    if disallowed_char in peptide:
                        is_valid = False
                        break
                if is_valid and len(peptide) <= max_length and peptide not in peptides:
                    peptides.add(peptide)

    return sorted(peptides)[:100]
                    
                    
def peptides_to_ms2pip_format(peptides, modifications=None):
    """
    Convert simple peptide sequences to the MS2PIP notation.
    
    Modifications can be specified by defining character substitutions.
    
    Args:
        peptides: A list of peptides to be converted to the MS2PIP notation.
        modifications: A dictionary of `mod_key`: (`mod_residue`, `mod_ms2pip`, `mod_mass`) for which `mod_key` 
                       is the character used to specify the modification, `mod_residue` is the residue on which the 
                       (static) modification is active, `mod_ms2pip` is the PTM name for MS2PIP, and `mod_mass` is 
                       the mass of the PTM (default: None).
        
    Returns:
        A list of peptides in the MS2PIP format.
    """
    peptides_ms2pip_format = []
    for peptide in tqdm.tqdm(peptides, desc='Peptides converted', unit='peptides'):
        peptide_ms2pip = 'H-{}-OH'.format(peptide)
        if modifications is not None:
            for mod_key, (mod_residue, _, _) in modifications.items():
                peptide_ms2pip = peptide_ms2pip.replace(mod_residue, mod_key + mod_residue)
        peptides_ms2pip_format.append(peptide_ms2pip)
    
    return peptides_ms2pip_format
        

def generate_ms2pip_spectra(peptides, fragmentation='CID', charge=2, modifications=None, chunk_size=10000, retry=10):
    """
    Use MS2PIP to generate simulated spectra for the given peptides.
    
    Args:
        peptides: The peptides (in MS2PIP format) for which to generate simulated spectra.
        modifications: A dictionary of `mod_key`: (`mod_residue`, `mod_ms2pip`, `mod_mass`) for which `mod_key` 
                       is the character used to specify the modification, `mod_residue` is the residue on which the 
                       (static) modification is active, `mod_ms2pip` is the PTM name for MS2PIP, and `mod_mass` is 
                       the mass of the PTM (default: None).
        fragmentation: The type of fragmentation to fragment the spectra; either CID or HCD (default: CID).
        charge: The charge of the spectrum; either 2 or 3 (default: 2).
        chunk_size: The number of peptides to process in batch to not overload the MS2PIP server (default: 10,000).
        retry: The number of times the request to the MS2PIP server is retried in case of a failure.
    
    Returns:
        A list containing tuples with the following information for each generated spectrum:
            - The peptide sequence
            - The peptide's mass over charge
            - A list representing the spectrum with tuples of (mz, intensity, annotation)
    """
    aa_mass = mass.std_aa_mass
    ms2pip_ptm = []
    if modifications is not None:
        for mod_key, (_, mod_ms2pip, mod_mass) in modifications.items():
            # add the modifications to the mass calculation
            aa_mass[mod_key] = mod_mass
            # and to the MS2PIP request
            ms2pip_ptm.append({'symbol': mod_key, 'ptm': mod_ms2pip})
        mod_chars = ''.join(modifications.keys())
    else:
        mod_chars = ''
    
    url = 'http://iomics.ugent.be/ms2pip/ms2pip-{}-api'.format(fragmentation)
    headers = {'Content-type': 'application/json'}

    # process the peptides in (limited) chunks to not overload the MS2PIP server
    spectra = []
    for peptide_chunk in tqdm.tqdm(chunks(peptides, chunk_size), desc='Chunks processed', total=int(math.ceil(float(len(peptides)) / chunk_size)), unit='chunks'):
        data = {'ptms': ms2pip_ptm, 'peptides': [{'charge': charge, 'peptide': peptide} for peptide in peptide_chunk]}

        # convert input to json for REST post
        json_data = json.dumps(data)

        times_tried = 0
        if times_tried < retry:
            try:
                # issue the request to the server
                response = requests.post(url, data=json_data, headers=headers)
                
                # convert the predicted spectra to a useful format
                predicted_spectra = json.loads(response.content)
            except ValueError:
                print('Request failed: {}'.format(response.content), file=sys.stderr)
                times_tried += 1
        else:
            print('Failed to issue the request to the MS2PIP server', file=sys.stderr)
            continue

        for peptide_key, peptide_spectrum in predicted_spectra.items():
            peptide_sequence = peptide_key[peptide_key.find('-') + 1: peptide_key.rfind('-')].encode('ascii', 'ignore')
            peptide_mz = mass.fast_mass(peptide_sequence, charge=charge, aa_mass=aa_mass)
            # remove non-standard characters denoting PTMs
            peptide_sequence = peptide_sequence.translate(None, mod_chars)
            
            peaks = []
            for peak_idx in range(len(peptide_spectrum['peaks'])):
                peak_mz = peptide_spectrum['mz'][peak_idx]
                peak_intensity = 10000 * (2 ** peptide_spectrum['peaks'][peak_idx] - 0.001)
                if peak_intensity > 0.001:
                    ions = peptide_spectrum['ions'][peak_idx]
                    peak_annotation = '{}{}^{}'.format(ions[0], ions[ions.rfind('.') + 1:], ions[2])
                    
                    peaks.append((peak_mz, peak_intensity, peak_annotation))
            
            spectra.append((peptide_sequence, peptide_mz, sorted(peaks, key=operator.itemgetter(0))))
    
    return spectra


def chunks(long_list, chunk_size):
    for i in range(0, len(long_list), chunk_size):
        yield long_list[i: i + chunk_size]

In [ ]:
# create a complete simulated spectral library containing target and decoy sequences
def simulate_spectra(fasta_filename, min_length, max_length, protease='trypsin', missed_cleavages=0,
                     modifications=None, fragmentation='CID', charge=2):
    """
    Generate a simulated spectral library using MS2PIP.
    
    Args:
        fasta_filename: The fasta file containing the protein sequences.
        min_length: The minimum length of the generated peptides (inclusive).
        max_length: The maximum length of the generated peptides (inclusive).
        protease: The protease used to cleave the proteins into peptides (default: trypsin).
        missed_cleavages: The number of missed cleavages allowed (default: 0).
        modifications: A dictionary of `mod_key`: (`mod_residue`, `mod_ms2pip`, `mod_mass`) for which `mod_key` 
                       is the character used to specify the modification, `mod_residue` is the residue on which the 
                       (static) modification is active, `mod_ms2pip` is the PTM name for MS2PIP, and `mod_mass` is 
                       the mass of the PTM (default: None).
        fragmentation: The type of fragmentation to fragment the spectra; either CID or HCD (default: CID).
        charge: The charge of the spectrum; either 2 or 3 (default: 2).
    
    Returns:
        A list of simulated spectra for all peptides derived from the given fasta file, with the following 
        information for each peptide:
            - The peptide sequence
            - The peptide's mass over charge
            - A list representing the spectrum with tuples of (mz, intensity, annotation)
    """
    peptides = peptides_to_ms2pip_format(
        fasta_to_peptides(fasta_filename, min_length, max_length, protease, missed_cleavages), modifications)
    
    return generate_ms2pip_spectra(peptides, fragmentation, charge, modifications)
                    
                    
def write_spectrum(out, spectrum, spec_id, charge, is_decoy):
    """
    Write a spectrum to the output stream.
    
    Args:
        out: The output stream to write to.
        spectrum: The spectrum to be written.
        spec_id: The (unique) identifier for this spectrum.
        charge: The spectrum's charge.
        is_decoy: Flag indicating whether the spectrum is a target spectrum or a decoy spectrum.
    """
    sequence, mz, peaks = spectrum
    
    out.write('Name: {}/{}\n'.format(sequence, charge))
    out.write('LibID: {}\n'.format(spec_id))
    out.write('MW: {}\n'.format(mz * charge))
    out.write('PrecursorMZ: {}\n'.format(mz))
    out.write('Status: Normal\n')
    out.write('Fullname: X.{}.X/{}\n'.format(sequence, charge))
    out.write('Comment: {}\n'.format('None' if not is_decoy else 'Remark=DECOY_{}'.format(sequence)))

    out.write('NumPeaks: {}\n'.format(len(peaks)))
    for peak in peaks:
        out.write('{}\t{}\t{}\n'.format(*peak))

    out.write('\n')

def simulated_target_decoy_spectral_library(target_fasta, decoy_fasta, sptxt_out, min_length, max_length,
                                            protease='trypsin', missed_cleavages=0, modifications=None,
                                            fragmentation=('CID',), charge=(2,)):
    """
    Generated a simulated spectral library containing target and decoy sequences and write it to an .sptxt file.
    
    Args:
        target_fasta: The fasta file containing the target protein sequences.
        decoy_fasta: The fasta file containing the decoy protein sequences.
        sptxt_out: The filename to which the spectral library will be written in .sptxt format.
        min_length: The minimum length of the generated peptides (inclusive).
        max_length: The maximum length of the generated peptides (inclusive).
        protease: The protease used to cleave the proteins into peptides (default: trypsin).
        missed_cleavages: The number of missed cleavages allowed (default: 0).
        modifications: A dictionary of `mod_key`: (`mod_residue`, `mod_ms2pip`, `mod_mass`) for which `mod_key` 
                       is the character used to specify the modification, `mod_residue` is the residue on which the 
                       (static) modification is active, `mod_ms2pip` is the PTM name for MS2PIP, and `mod_mass` is 
                       the mass of the PTM (default: None).
        fragmentation: An iterable of fragmentation types to fragment the spectra; either CID or HCD are allowed 
                       (default: ('CID',).
        charge: An iterable of the charges of the spectrum; either 2 or 3 are allowed (default: (2,)).
    """
    with open(sptxt_out, 'w') as sptxt_out:
        idx = 0
        # multiple fragmentation types and charges might be possible
        for frag in fragmentation:
            for c in charge:
                # target spectra
                print('Generate target spectra for {} fragmentation and charge {}'.format(frag, c), file=sys.stderr)
                target_sequences = set()
                for spectrum in simulate_spectra(target_fasta, min_length, max_length, protease,
                                                 missed_cleavages, modifications, frag, c):
                    write_spectrum(sptxt_out, spectrum, idx, c, False)
                    idx += 1
                    target_sequences.add(spectrum[0])
                # decoy spectra
                print('Generate decoy spectra for {} fragmentation and charge {}'.format(frag, c), file=sys.stderr)
                for spectrum in simulate_spectra(decoy_fasta, min_length, max_length, protease,
                                                 missed_cleavages, modifications, frag, c):

                    if spectrum[0] not in target_sequences:    # don't add identical target and decoy spectra
                        write_spectrum(sptxt_out, spectrum, idx, c, True)
                        idx += 1


## iPRG 2012

The first dataset was generated in the context of the iPRG 2012 study on detecting modified peptides. \cite{Chalkley2014}

### Dataset

For this study a mixture of synthetic post-translationally modified peptides were spiked into a yeast lysate background. Tandem mass spectra were acquired on an AB SCIEX TripleTOF 5600 mass spectrometer.

All data for the iPRG 2012 study were retrieved from the [MassIVE](http://massive.ucsd.edu/ProteoSAFe/result.jsp?task=48e9a924f0394d6d8d1ed04a1716e73b&view=advanced_view) data repository.

As specified by Chalkley et al. \cite{Chalkley2014}, a specific protein sequence database was provided for this study. The protein sequence database was created from UniprotKB and was filtered to contain all yeast proteins, the bovine
protein PDIA1, all human proteins, and TRYP_PIG, yielding a total of
42,450 sequences.

Further, a decoy database was generated by scrambling the sequence of amino acids while holding the position of (1) all internal lysines and arginines, and (2) prolines followed by a lysine or arginine, fixed.

### Search settings

The following settings were used to generate the *in silico* peptides:

* Peptide length between 8 and 40 (inclusive) amino acids.
* Trypsin cleavage with at most two missed cleavages.
* No identical peptides present in both the target and decoy sets of peptides.

This results in a set of 2,783,549 unique target peptide sequences and 2,925,357 unique decoy peptide sequences (613 sequences overlapped between the target set and the decoy set and were only retained in the target set).

The following settings were used to generate simulated spectra for each peptide:

* Spectra with a charge of both 2 and 3 were generated, according to CID fragmentation.
* A static carbamidomethyl modification (+57.021464 Da) of cysteines.

The resulting 11,417,812 spectra were exported to a spectral library file in the [.sptxt format](http://tools.proteomecenter.org/wiki/index.php?title=Software:SpectraST#sptxt_file_format:) and subsequently converted to the custom SQLite-based .spql format.

In [ ]:
# create a simulated spectral library for the iPRG 2012 study
simulated_target_decoy_spectral_library(target_fasta='../data/external/ABRF_iPRG_2012_target.fasta',
                                        decoy_fasta='../data/external/ABRF_iPRG_2012_decoy.fasta',
                                        sptxt_out='../data/interim/ABRF_iPRG_2012_MS2PIP_targetdecoy.sptxt',
                                        min_length=8, max_length=40,
                                        protease='trypsin', missed_cleavages=2,
                                        modifications={'c': ('C', 'Carbamidomethyl', 57.021464)},
                                        fragmentation=('CID',), charge=(2, 3))

In [ ]:
# convert the generated spectral library to .spql format
convert_spectrast_to_spql.convert(spectrast_filename='../data/interim/ABRF_iPRG_2012_MS2PIP_targetdecoy.sptxt',
                                  spql_filename='../data/processed/ABRF_iPRG_2012_MS2PIP_targetdecoy.spql')

## Chick et al.

The second dataset was used by Chick et al. to investigate the source of unassigned spectra. \cite{Chick2015}

### Dataset

For this study a proteome-wide dataset from HEK293 cells was collected. High-resolution and high-mass accuracy MS/MS spectra were collected using a Thermo Scientific Q-Exactive Orbitrap mass spectrometer.

All spectral data for this study were retrieved from the [PRIDE](https://www.ebi.ac.uk/pride/archive/projects/PXD001468) data repository. (**TODO**: In what format? RAW or mzXML? Specify.)

Similar to settings used by Chick et al. \cite{Chick2015}, the Homo_sapiens GRCh37.61 fasta file containing the protein translations of human gene predictions was retrieved from [Ensembl](http://ftp.ensembl.org/pub/release-61/fasta/homo_sapiens/pep/). This fasta file contains 86,934 protein sequences in total.

Using this fasta file a decoy database was created by generating reversed sequences using Pyteomics. \cite{Goloborodko2013}

### Search settings

The following settings were used to generate the *in silico* peptides:

* Peptide length between 8 and 40 (inclusive) amino acids.
* Trypsin cleavage with at most one missed cleavage.
* No identical peptides present in both the target and decoy sets of peptides.

This results in a set of 1,413,848 unique target peptide sequences and 1,415,572 unique decoy peptide sequences (174 sequences overlapped between the target set and the decoy set and were only retained in the target set).

The following settings were used to generate simulated spectra for each peptide:

* Spectra with a charge of both 2 and 3 were generated, according to HCD fragmentation.
* A static carbamidomethyl modification (+57.021464 Da) of cysteines.

The resulting 5,658,840 spectra were exported to a spectral library file in the [.sptxt format](http://tools.proteomecenter.org/wiki/index.php?title=Software:SpectraST#sptxt_file_format:) and subsequently converted to the custom SQLite-based .spql format.

In [ ]:
# create a separate decoy fasta file
fasta.write_decoy_db(source='../data/external/Homo_sapiens.GRCh37.61.pep.all_target.fasta',
                     output='../data/interim/Homo_sapiens.GRCh37.61.pep.all_decoy.fasta',
                     mode='reverse', prefix='DECOY_', decoy_only=True)

In [ ]:
# create a proteome-wide human simulated spectral library
simulated_target_decoy_spectral_library(target_fasta='../data/external/Homo_sapiens.GRCh37.61.pep.all_target.fasta',
                                        decoy_fasta='../data/interim/Homo_sapiens.GRCh37.61.pep.all_decoy.fasta',
                                        sptxt_out='../data/interim/Homo_sapiens.GRCh37.61.pep.all_MS2PIP_targetdecoy.sptxt',
                                        min_length=8, max_length=40,
                                        protease='trypsin', missed_cleavages=1,
                                        modifications={'c': ('C', 'Carbamidomethyl', 57.021464)},
                                        fragmentation=('HCD',), charge=(2, 3))

In [ ]:
# convert the generated spectral library to .spql format
convert_spectrast_to_spql.convert(spectrast_filename='../data/interim/Homo_sapiens.GRCh37.61.pep.all_MS2PIP_targetdecoy.sptxt',
                                  spql_filename='../data/processed/Homo_sapiens.GRCh37.61.pep.all_MS2PIP_targetdecoy.spql')

# References

[<a id="cit-Griss2016" href="#call-Griss2016">1</a>] Griss Johannes, ``_Spectral Library Searching in Proteomics_'', PROTEOMICS, vol. 16, number 5, pp. 729--740, March 2016.

[<a id="cit-Shao2016" href="#call-Shao2016">2</a>] Shao Wenguang and Lam Henry, ``_Tandem Mass Spectral Libraries of Peptides and their Roles in Proteomics Research_'', Mass Spectrometry Reviews, vol. , number , pp. ,  .

[<a id="cit-Zhang2011" href="#call-Zhang2011">3</a>] Zhang Xin, Li Yunzi, Shao Wenguang <em>et al.</em>, ``_Understanding the Improved Sensitivity of Spectral Library Searching over Sequence Database Searching in Proteomics Data Analysis_'', PROTEOMICS, vol. 11, number 6, pp. 1075--1085, March 2011.

[<a id="cit-Yen2009" href="#call-Yen2009">4</a>] Yen Chia-Yu, Meyer-Arendt Karen, Eichelberger Brian <em>et al.</em>, ``_A Simulated MS/MS Library for Spectrum-to-Spectrum Searching in Large Scale Identification of Proteins_'', Molecular \& Cellular Proteomics, vol. 8, number 4, pp. 857--869, April 2009.

[<a id="cit-Yen2011" href="#call-Yen2011">5</a>] Yen Chia-Yu, Houel Stephane, Ahn Natalie G <em>et al.</em>, ``_Spectrum-to-Spectrum Searching Using a Proteome-Wide Spectral Library_'', Molecular \& Cellular Proteomics, vol. 10, number 7, pp. M111.007666--M111.007666, July 2011.

[<a id="cit-Degroeve2013" href="#call-Degroeve2013">6</a>] Degroeve Sven and Martens Lennart, ``_MS2PIP: A Tool for MS/MS Peak Intensity Prediction_'', Bioinformatics, vol. 29, number 24, pp. 3199--3203, September 2013.

[<a id="cit-Degroeve2015" href="#call-Degroeve2015">7</a>] Degroeve Sven, Maddelein Davy and Martens Lennart, ``_MS2PIP Prediction Server: Compute and Visualize MS2 Peak Intensity Predictions for CID and HCD Fragmentation_'', Nucleic Acids Research, vol. 43, number W1, pp. W326--W330, July 2015.

[<a id="cit-Goloborodko2013" href="#call-Goloborodko2013">8</a>] Goloborodko Anton A, Levitsky Lev I, Ivanov Mark V <em>et al.</em>, ``_Pyteomics-a Python Framework for Exploratory Data Analysis and Rapid Software Prototyping in Proteomics_'', Journal of The American Society for Mass Spectrometry, vol. 24, number 2, pp. 301--304, February 2013.

[<a id="cit-Chalkley2014" href="#call-Chalkley2014">9</a>] Chalkley Robert J, Bandeira Nuno, Chambers Matthew C <em>et al.</em>, ``_Proteome Informatics Research Group (iPRG)\_2012: A Study on Detecting Modified Peptides in a Complex Mixture_'', Molecular \& Cellular Proteomics, vol. 13, number 1, pp. 360--371, January 2014.

[<a id="cit-Chick2015" href="#call-Chick2015">10</a>] Chick Joel M, Kolippakkam Deepak, Nusinow David P <em>et al.</em>, ``_A Mass-Tolerant Database Search Identifies a Large Proportion of Unassigned Spectra in Shotgun Proteomics as Modified Peptides_'', Nature Biotechnology, vol. 33, number 7, pp. 743--749, June 2015.

